In [53]:
import pandas as pd
from langchain_groq import ChatGroq
import json

In [6]:
df = pd.read_csv("Team Anantam Recruitment Registration  (Responses) - Form Responses 1.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 17 columns):
 #   Column                                                                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                                                                           --------------  ----- 
 0   Timestamp                                                                                                                                                        85 non-null     object
 1   Full Name                                                                                                                                                        85 non-null     object
 2   Email                                                                                                                                   

In [8]:
# Drop specified columns
columns_to_drop = [
    'Timestamp',
    'Email',
    'Mobile No ( WhatsApp )',
    'Your Work related to Rocketry and Astronomy. If any; you can mention ( Attended workshop, certificate, videos, Astrophotography, anything that differentiates )'
]

df = df.drop(columns=columns_to_drop)


KeyError: "['Timestamp', 'Email', 'Mobile No ( WhatsApp )', 'Your Work related to Rocketry and Astronomy. If any; you can mention ( Attended workshop, certificate, videos, Astrophotography, anything that differentiates )'] not found in axis"

In [22]:
df.head()


,Full Name,Academic Year,Branch/Department,Select the Domain you are Interested in,"You are Intersted in Anantam Rocketry, what motivates you ? Why is this field intersting to you ? ( ( None if not Applicable)","You are Intersted in Anantam Astronomy, what motivates you ? Why is this field intersting to you ? (None if not Applicable)",How do you think you can contribute to the team ?,"Describe yourself in 3 words, Justify your interests.",(Anantam Rocketry) Please Select the domains you are Interested in (Select None if not Applicable),(Anantam Astronomy) Please Select the domains you are Interested in (Select None if not Applicable),"What do you think; is this field important, especially for India ?","We are a new team, If you were the Leader what will be your approach ?","Are you already a part of any Depatment Club, Motorsports team ?"
0,Harshal Rajesh More,SY,EnTC,"Anantam Rocketry- Rocket design, Propulsion, b...","Space was always a field of interest for me, e...",None,I have a done a course in rocket designing(off...,Hardworking\nCurious \nAdaptable,"Design and Aerodynamics, Avionics and Communic...",None,With our recent achievement of India being the...,The first part will be setting up things like ...,No
1,Vedant Nandkishor Bharsakle,Final Year,Mechanical,"Anantam Rocketry- Rocket design, Propulsion, b...",One of my great juniors has started a team. Wh...,-,Probably can share some of my experience at IS...,Keen to learn \nAerospace/ aeronautics enthusi...,"Design and Aerodynamics, Structure and Analysi...",None,I'll tell you the objectives why ISRO does wha...,Finding a core team... doesn't matter if they ...,Yes
2,Vangar Prathmesh Shriniwas,SY,Mechanical,"Anantam Rocketry- Rocket design, Propulsion, b...",I am always interested in new technology .I ca...,I like see star new things discovered in space.,I can do any thing for team .,Punctual Confident Passionate. My interest is ...,"Design and Aerodynamics, Structure and Analysi...",Event Management,Yes,"As a leader, i will try to approach each team ...",I worked with Mr. Hemant Kadam sir for his PhD...
3,Vedant Nandedkar,TY,ENTC,"Anantam Rocketry- Rocket design, Propulsion, b...",I have been interested in space since I was yo...,None.,I can help in organising and doing reasearch a...,"Rocket, Space and Anantum","Design and Aerodynamics, Avionics and Communic...",Public Relations,"Yes ,it is important as space and every commun...",I would first help organise and create basic p...,I was part of Team Kratos racing for 12 months.
4,Tejashree Danling Chougule,TY,CSE AIML,"Anantam Rocketry- Rocket design, Propulsion, b...",I'm fascinated by Anantam Rocketry because it ...,As it allows me to connect with the vastness o...,My dedication to continuous learning and teamw...,"Curious, Analytical, Determined. My interest i...","Design and Aerodynamics, Automation and ML","Event Management, Marketing, Public Relations","Yes, this field is crucial for India as it pro...","As a leader, my approach would focus on foster...",Yes


In [16]:
all_columns = df.columns.tolist()


'Full Name'

In [23]:
question_answer_list = []

for index, row in df.iterrows():
    questions_answers = ""
    for column in all_columns:
        question = column
        answer = row[column]
        questions_answers += f"Question: {question}\nAnswer: {answer}\n\n"
    
    question_answer_list.append(questions_answers)


In [ ]:
llm = ChatGroq(api_key=GROQ_API_KEY,
               model='llama-3.1-70b-versatile',
               temperature=0.3)

In [54]:
response_list = []

for i in question_answer_list:

    prompt_template = f"""
    You are a recruiter for a company. You are given a set of interview questions and answers of a candidate and you need
    to provide a score between 0 and 100 based on the answers. Give a very precise score. Try to give a different and 
    precise score for each candidate. Also provide a brief explanation for the score.

    The set of questions and answers are as follows:
    {i}

    The output should be strictly in a JSON format as follows:-
    (open curly bracket)
        "name": "name of the candidate",
        "score": "0-100",
        "explanation": "brief explanation for the score"
    (close curly bracket)

    Output should be in JSON format only. Do not include any other text.
    """

    response = llm.invoke(prompt_template)
    try:
        response_dict = json.loads(response.content)
    except json.JSONDecodeError:
        print(f"Failed to parse JSON for response: {response.content}")
        continue
    response_list.append(response_dict)

In [55]:
response_list

[{'name': 'Harshal Rajesh More',
  'score': '87.2',
  'explanation': 'The candidate has shown a strong interest in Anantam Rocketry and has relevant experience in rocket designing and electronics. They have also demonstrated a passion for space exploration and its importance for India. Their leadership approach is well-structured and inclusive. However, their answers could be more detailed and specific, and they lack experience in being part of a department club or motorsports team.'},
 {'name': 'Vedant Nandkishor Bharsakle',
  'score': '74',
  'explanation': 'Vedant has shown a keen interest in Anantam Rocketry and has relevant experience to share from ISRO. He has also demonstrated leadership skills by outlining an approach to building a core team. However, his answers lack depth and clarity in some areas, such as explaining why the field is important for India. Additionally, his motivation for joining the team seems to be influenced by a personal connection rather than a genuine pas

In [59]:
# Sort the response_list in descending order based on the score
sorted_response_list = sorted(response_list, key=lambda x: float(x['score']), reverse=True)

# Display the sorted list
sorted_response_list
# Store the sorted list in a JSON file

json_filename = 'sorted_candidates.json'

with open(json_filename, 'w') as json_file:
    json.dump(sorted_response_list, json_file, indent=4)

print(f"Sorted candidate list has been saved to {json_filename}")


Sorted candidate list has been saved to sorted_candidates.json
